# Homework 1: Implement TOPSIS method and entropy weight method

## 1. TOPSIS Method: Use the data of research institutions

In [1]:
import numpy as np
from copy import deepcopy

In [2]:
### Load data and weights
A = [0.1, 5, 5000, 4.7]
B = [0.2, 6, 6000, 5.6]
C = [0.4, 7, 7000, 6.7]
D = [0.9, 10, 10000, 2.3]
E = [1.2, 2, 400, 1.8]
data = [A, B, C, D, E]
### pubishment, ratio of students to teachers, funds, deferred graduation rate
weights = [0.2, 0.3, 0.4, 0.1]

### Analysis:
- For the four criteria, publishment and funds are large preferred, the ratio is small preferred, funds are preferred in a particular bound.
- For the criterion of the ratio of students to teachers, we assume the best field would be [5,6], ratio under 2 or over 12 (inclusive) will get score of 0.
- For the criterion of deferred, we takes the reciprocal of it.

In [3]:
### process the ratio data and deffered graduation rate data.
def range2max(rangevalue, num):
    lowest, highest, best_range_left, best_range_right = deepcopy(rangevalue)
    if num >= best_range_left and num <= best_range_right:
        return 1
    elif num < best_range_left:
        return (num - lowest) / (best_range_left - lowest)
    elif num > best_range_right:
        return (highest - num) / (highest - best_range_right)
    else:
        return 0

def min2max(num):
    return 1/num

In [4]:
def process_data(data):
    ### fist determine the values
    columns = [1, 3, ...] # columns needed to merge
    rangevalue = [(2, 12, 5, 6), ...] # lowest, highest, best_range_left, best_range_right
    ### Merge the data.
    m_data = deepcopy(data)
    for row in m_data:
        row[columns[0]] = range2max(rangevalue[0], row[columns[0]])
        row[columns[1]] = min2max(row[columns[1]])
    m_data = np.array(m_data)
    ### Normalize the data into [0,1]
    norm_data = deepcopy(m_data)
    for i in range(norm_data.shape[1]):
        norm_data[:, i] = norm_data[:, i]/np.sqrt(np.sum(norm_data[:, i]**2))
    ### Get ideal data in normalized real data
    ideal_bad = [min(norm_data[:, i]) for i in range(norm_data.shape[1])]
    ideal_good = [max(norm_data[:, i]) for i in range(norm_data.shape[1])]
    ### Calculate the distance between real data and ideal data
    distance_plus = np.array([])
    distance_minus = np.array([])
    for i in range(norm_data.shape[0]):
        distance_plus = np.append(distance_plus, np.sqrt(np.sum(np.dot((norm_data[i] - ideal_good)**2, weights))))
        distance_minus = np.append(distance_minus, np.sqrt(np.sum(np.dot((norm_data[i] - ideal_bad)**2, weights))))
    print("=========distances============")
    print("distance_plus: {}".format(distance_plus))
    print("distance_minus: {}".format(distance_minus))
    print("=========final scores============")
    ### Calculate the score
    scores = distance_minus / (distance_minus + distance_plus)
    for i in range(norm_data.shape[0]):
        print("The score of school %s is %f" % (chr(65 + i), scores[i]))

process_data(data)

=========distances============
distance_plus: [0.40698357 0.36817733 0.31587852 0.23934202 0.53135666]
distance_minus: [0.38455211 0.40935987 0.40556474 0.50296763 0.3549938 ]
=========final scores============
The score of school A is 0.485830
The score of school B is 0.526483
The score of school C is 0.562158
The score of school D is 0.677571
The score of school E is 0.400512


#### Now every criterion has turned into large-preferred data.


## 2. Entropy Weight Method

In [5]:
### Load data
A = [100, 90, 100, 84, 90, 100, 100, 100, 100]
B = [100, 100, 78.6, 100, 90, 100, 100, 100, 100]
C = [75, 100, 85.7, 100, 90, 100, 100, 100, 100]
D = [100, 100, 78.6, 100, 90, 100, 94.4, 100, 100]
E = [100, 90, 100, 100, 100, 90, 100, 100, 80]
F = [100, 100, 100, 100, 90, 100, 100, 85.7, 100]
G = [100, 100, 78.6, 100, 90, 100, 55.6, 100, 100]
H = [87.5, 100, 85.7, 100, 100, 100, 100, 100, 100]
I = [100, 100, 92.9, 100, 80, 100, 100, 100, 100]
J = [100, 90, 100, 100, 100, 100, 100, 100, 100]
K = [100, 100, 92.9, 100, 90, 100, 100, 100, 100]
data = [A, B, C, D, E, F, G, H, I, J, K]
data = np.array(data)
data

array([[100. ,  90. , 100. ,  84. ,  90. , 100. , 100. , 100. , 100. ],
       [100. , 100. ,  78.6, 100. ,  90. , 100. , 100. , 100. , 100. ],
       [ 75. , 100. ,  85.7, 100. ,  90. , 100. , 100. , 100. , 100. ],
       [100. , 100. ,  78.6, 100. ,  90. , 100. ,  94.4, 100. , 100. ],
       [100. ,  90. , 100. , 100. , 100. ,  90. , 100. , 100. ,  80. ],
       [100. , 100. , 100. , 100. ,  90. , 100. , 100. ,  85.7, 100. ],
       [100. , 100. ,  78.6, 100. ,  90. , 100. ,  55.6, 100. , 100. ],
       [ 87.5, 100. ,  85.7, 100. , 100. , 100. , 100. , 100. , 100. ],
       [100. , 100. ,  92.9, 100. ,  80. , 100. , 100. , 100. , 100. ],
       [100. ,  90. , 100. , 100. , 100. , 100. , 100. , 100. , 100. ],
       [100. , 100. ,  92.9, 100. ,  90. , 100. , 100. , 100. , 100. ]])

In [6]:
### Calculate weights from 0 to 1
### integrate the method into a function
def calc_weights(data):
    ### first normalize the data
    for i in range(data.shape[1]):
        data[:, i] = (data[:, i] - min(data[:, i])) / (max(data[:, i]) - min(data[:, i]))
    norm_data = data
    ### calculate possibilities
    possibilities = np.zeros((data.shape[0], data.shape[1]))
    prob_log_prob = np.zeros((data.shape[0], data.shape[1]))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            possibilities[i, j] = norm_data[i, j] / np.sum(norm_data[:, j])
            prob_log_prob[i, j] = possibilities[i, j] * np.log(possibilities[i, j]) if possibilities[i, j] != 0 else 0
    entropy = np.zeros(data.shape[1])
    ### To handle the zero case, we use the prob_log_prob to calculate the entropy
    for i in range(data.shape[1]):
        entropy[i] = -np.sum(prob_log_prob[:, i])/np.log(data.shape[0])
    duplicates = 1 - entropy
    weights = duplicates / np.sum(duplicates)
    return weights
weights = calc_weights(data)
print("The weights are: {}".format(weights))


The weights are: [0.07578559 0.2191587  0.27137381 0.06559212 0.1051977  0.06559212
 0.06611572 0.06559212 0.06559212]
